# Differential Privacy for PRETSA

In [6]:
!git clone https://github.com/cmalvr/PRETSA

fatal: destination path 'PRETSA' already exists and is not an empty directory.


In [7]:
!pip install -r /content/PRETSA/requirements.txt

In [8]:
!rm -r "/content/PRETSA/baselogs"

In [9]:
from pathlib import Path
import zipfile

# Define paths
zip_path = Path("/content/PRETSA/baselogs.zip")
extract_dir = Path("/content/PRETSA")

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Search for CSV files
csv_files = list(extract_dir.rglob("*.csv"))

if csv_files:
    print("Found CSV files:", [f.name for f in csv_files])
else:
    print("No CSV files found.")

Found CSV files: ['Sepsis_dataset_duration.csv', 'bpic2013_dataset.csv', 'Sepsis_dataset.csv', 'traffic_fines_dataset.csv', 'HospitalBilling_dataset.csv', 'CoSeLoG_dataset.csv']


In [10]:
!python /content/PRETSA/add_annotation_duration.py Sepsis_dataset /content/PRETSA/baselogs/Sepsis_dataset.csv

/content/PRETSA/runlogs/Sepsis_dataset_duration.csv


In [ ]:
!python /content/PRETSA/runPretsa.py <fileName> <k> <t>

In [ ]:
!python /content/PRETSA/generate_baseline_log.py <fileName> <k> <t>

In [ ]:
!python /content/PRETSA/calculateDatasetStatistics.py <fileName>
!python /content/PRETSA/calculatBaselineEventLogStatistics.py <dictName>
!python /content/PRETSA/calculatePRETSAEventLogStatistics.py <dictName>

In [ ]:
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py <dictName>
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py <dictName>

In [ ]:
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py <dictName>
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py <dictName>

In [ ]:
!python /content/PRETSA/generateAnnotationOriginalDataset.py <dictName>

In [ ]:
!python /content/PRETSA/calculateAnnotationError.py <dictName>